**Please write a generic python function (in a separate package) which implements a generic version of the Gaussian Mixture Models algorithm. Your function should take as input a $d$-dimensional data set and the number of clusters into which you wish to group your data set. Return the number of members, the means and covariances of your K-clusters. Also return the responsibilites of your observations (of every sample in your dataset). Use your K-means function to intialize your GMM.**

In [3]:
import numpy as np
from kmeans import *
from importer import *
from pprint import pprint

X, y = gmm_data()
d = X.shape[0]
N = X.shape[1]
l = np.random.randint(0,2,(N,))
C = list(set(l))
K = len(C)
max_iter = 1

# # Initialize with Kmeans
# u, labels = kmeans(X, K=2, max_iter=100)

# # Initialize Randomly
# u = []
# labels = np.random.randint(0, 2, N)
# for k in range(K):
#       u.append(X[:,k])
# pprint(u)


# Initialize Standard
labels1 = np.zeros(500)
labels2 = np.ones(500)
labels = np.concatenate((labels1, labels2))

Nj = []
Pj = []
Uj = []
Sj = []
for c, k in zip(C, range(K)):
    Nj.append(np.sum(labels==c))
    Pj.append(Nj[k]/N)
    Uj.append(1/Nj[k] * np.sum(X[:,labels==c], axis=1)[:,np.newaxis])
    Sj.append(1/Nj[k] * (X[:,labels==c] - Uj[k]) @ (X[:,labels==c] - Uj[k]).T)

for i in range(max_iter):
    # Expectation Step
    # PjGj = []
    # PjGj_sum = 0
    # Yj = []
    # for c, k in zip(C, range(K)):
    #     PjGj.append(Pj[k] * 1/np.sqrt(np.linalg.det(2*np.pi*Sj[k])) * np.exp(-0.5*(X-Uj[k]).T @ np.linalg.inv(Sj[k]) @ (X-Uj[k])))
    #     PjGj_sum += PjGj[k]
    # for c, k in zip(C, range(K)):
    #     Yj.append(PjGj[k] / PjGj_sum)

    PnjGnj = []
    PnjGnj_sum = 0
    Ynj = []
    for c, k in zip(C, range(K)):
        PnGn = np.zeros((N))
        for n in range(N):
            PnGn[n] = Pj[k] * 1/np.sqrt(np.linalg.det(2*np.pi*Sj[k])) * np.exp(-0.5*(X[:,n,None]-Uj[k]).T @ np.linalg.inv(Sj[k]) @ (X[:,n,None]-Uj[k]))
        PnjGnj.append(PnGn)
        PnjGnj_sum += PnjGnj[k]
    for c, k in zip(C, range(K)):
        Ynj.append(PnjGnj[k] / PnjGnj_sum)

    # Maximization Step
    Nj = []
    Pj = []
    Uj = []
    Sj = []
    for c, k in zip(C, range(K)):
        Nj.append(np.sum(Ynj[k]))
        Pj.append(Nj[k]/N)
        Uj.append((1/Nj[k] * X @ Ynj[k][:,np.newaxis]))
        Sj.append(1/Nj[k] * (Ynj[k] * (X - Uj[k])) @ (X - Uj[k]).T)

    # Log-Likelihood
    PnjGnj = []
    PnjGnj_sum = 0
    for c, k in zip(C, range(K)):
        PnGn = np.zeros((N))
        for n in range(N):
            PnGn[n] = Pj[k] * 1/np.sqrt(np.linalg.det(2*np.pi*Sj[k])) * np.exp(-0.5*(X[:,n,None]-Uj[k]).T @ np.linalg.inv(Sj[k]) @ (X[:,n,None]-Uj[k]))
        PnjGnj.append(PnGn)
        PnjGnj_sum += PnjGnj[k]
    LL = np.sum(np.log(PnjGnj_sum))

Probabilities = Ynj[0]
for k in range(K-1):
    Probabilities = np.stack((Probabilities, Ynj[k+1]), axis=0)
labels = np.argmax(Probabilities, axis=0)

print(Ynj[0].shape)